In [10]:
# !pip install pandas pyarrow numpy matplotlib seaborn top2vec IProgress sentence-transformers bertopic nltk

In [12]:
!pip install tensorflow

In [13]:
import re

import pandas as pd
import numpy as np

from umap import UMAP
from hdbscan import HDBSCAN
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer

from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired
from bertopic.vectorizers import ClassTfidfTransformer

import ssl
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

import seaborn as sns
import matplotlib.pyplot as plt


try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

# nltk.download("stopwords")
russian_stopwords = stopwords.words("russian")

SyntaxError: invalid syntax (pywrap_tensorflow_internal.py, line 114)

In [ ]:
df = pd.read_csv("TEST.csv").drop("Unnamed: 0", axis=1)
df["timestamp"] = pd.to_datetime(df.timestamp)

In [ ]:
print(f"Размер данных: {df.shape[0]}")

df.head()

In [ ]:
df.role.value_counts()

In [ ]:
print(f"Количество уникальных сессий: {df.session_id.nunique()}")

Посмотрим пропуски

In [ ]:
df.isna().sum()

In [ ]:
df[df.session_id.isna()]

Есть один пропуск, удалим его

In [ ]:
df = df[~df.session_id.isna()]

Отсортируем данные по `timestamp` и выделим оттуда фичи

In [ ]:
df = df.sort_values("timestamp")

In [ ]:
df["date"] = df.timestamp.dt.date
df["day_of_week"] = df.timestamp.dt.day_of_week
df["hour"] = df.timestamp.dt.hour

In [ ]:
df.head(2)

In [ ]:
df.nunique()

Так, получается, что предоставленные данные урезаны до двух дней, тем не менее можем посмотреть распределение обращений по часам.

Нельзя сказать, что это будет репрезентативно, да еще и данные предоставлены сразу после Нового года

In [ ]:
sns.barplot(df[df.role == "user"].groupby("hour").session_id.nunique())

plt.title("Распределение сессий по часам")
plt.ylabel("quantity")

plt.show()

Можем заметить, что к чат-боту чаще всего обращаются в 9 часов, что примерно в начале рабочего дня. Возможно какие-то проблемы с запуском оборудования.

Но тем не менее обращения в течение дня так же есть. Скорее всего они связаны с какими-то поломками во время работы.

# **Кластеризация первых сообщений пользователей**

Посмотрим пример одной сессии

In [ ]:
df.groupby("session_id").text.apply(lambda x: list(x)).iloc[2]

**Нам нужно кластеризировать обращения пользователей.**

Здесь важный момент: мы хотим понять с какими запросами к нам приходят пользователи и провести какую-то аналитику или же, что более веротяно в контексте чат-бота, *мы хотим понимать к какому кластеру относится новый запрос пользователя и использовать это при построении ответа.*

Если говорить о первой задаче, то тут уместнее использовать весь контекст сессии, чтобы лучше выделить ключевые моменты и сделать хороший анализ.

**В продакшене же мы не можем знать заранее всю нашу сессию, поэтому нам нужно уметь хорошо кластерезировать по первому сообщения пользователя.**

Возможно нужно брать первые 2 сообщения пользователя, так как иногда первое сообщение носит характер приветственого, а второе уже для раскрытия проблемы, но не везде. 

Например здесь вообще нет второго сообщения от пользователя:

In [ ]:
df.groupby("session_id").text.apply(lambda x: list(x)).iloc[3]

**Пока что будет использовать только самое первое сообщение в сессии от пользователя**

Достанем первой сообщение из сессии от пользователя

In [ ]:
docs = df[df.role == "user"].groupby("session_id").text.apply(lambda x: list(x)[0]).tolist()
docs[:5]

## Baseline

**Пайплайн кластеризации:**

**Получение эмбеддингов обращений -> сокращение размерности -> кластеризация точек -> понимание что отличает кластеры друг от друга (BOW + c-TF-IDF)**

Будем использовать **BERTopic.** 

Параметры подберем позже

Вычислим заранее эмбеддинги для постов

In [ ]:
# embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
embedding_model = SentenceTransformer("distiluse-base-multilingual-cased-v1")
embeddings = embedding_model.encode(docs)

embeddings[:2]

In [ ]:
# Step 2 - Reduce dimensionality
umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine', random_state=1)

# Step 3 - Cluster reduced embeddings
hdbscan_model = HDBSCAN(min_cluster_size=15, metric='euclidean', cluster_selection_method='eom', prediction_data=True)

# Step 4 - Tokenize topics
vectorizer_model = CountVectorizer()

# Step 5 - Create topic representation
ctfidf_model = ClassTfidfTransformer()

In [ ]:
topic_model = BERTopic(
  embedding_model=embedding_model,
  umap_model=umap_model,
  hdbscan_model=hdbscan_model,
  vectorizer_model=vectorizer_model,
  ctfidf_model=ctfidf_model
)

topics, probs = topic_model.fit_transform(documents=processed_docs, embeddings=embeddings)

In [ ]:
topic_model.get_topic_info()

In [ ]:
df[df.text.str.contains("jpeg")].head(3)

In [ ]:
topic_model.get_representative_docs()

In [ ]:
fig = plt.figure(figsize=(15, 7))

topic_model.visualize_documents(docs)

Посмотрим на визуальное представления наших кластеров и сделаем парочку выводов:

* Некоторые пользователи не сразу пишут свою проблему: в первом сообщении они сначала пишут "Добрый день" или "Здравствуйте", а уже потом вероятно обозначают свой запрос **(Кластеры 2, 5)**. Некоторые же присылают первое сообщение "Оператор", видимо чтобы проверить, что бот работает **(Кластер 3)**
* Некоторые пользователи в первом же сообщении сессии уже выражают благодарность или присылают какой-то файл. Гипотеза в том, что вероятно у них есть доступ к предыдущей сессии, которая была завершена из-за молчания пользователя, но после пользователь все же разрешил эту проблему и отправил это сообщение, но уже в новую сессию **(Кластер 1, 6)**
* У других пользователей проблема с сим-картой **(Кластер 4)**
* Ну и основной кластер с наибольшим количеством объектов **(Кластер 0)**. В нем неудачно выделились ключевые слова, и мы не особо понимаем проблемы пользователей

## Tune

In [ ]:
df_topics = topic_model.get_document_info(docs)

print(f"Размер: {df_topics.shape[0]}")
df_topics.head(3)

**Для простоты оставим только Кластеры -1, 0, 4 и проанализируем их.**

In [ ]:
new_docs = df_topics[(df_topics["Topic"] == 0) | (df_topics["Topic"] == 4) | (df_topics["Topic"] == -1)]["Document"].tolist()
new_docs[:3]

In [ ]:
new_emb = embedding_model.encode(new_docs)
new_emb[:2]

Предобработаем текст, который будем использовать для вычленения ключевых слов, характерезующих кластер

In [ ]:
lemmatizer = WordNetLemmatizer()
noise_list = ["привет", "доброе утро", "добрый день", "добрый вечер", "здравствуйте", "до свидания", 
              "спасибо", "хорошо", "пожалуйста", "c новым годом"]

def remove_noise(text: str) -> str:
    for noise in noise_list:
        text = text.replace(noise, "")
    return text

def process_text(text: str, lemmatizer: WordNetLemmatizer = lemmatizer) -> str:
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    text = remove_noise(text)
    
    words = text.split()
    words = [word for word in words if word not in russian_stopwords]
    words = [lemmatizer.lemmatize(word) for word in words]
    
    return ' '.join(words)

In [ ]:
processed_new_docs = [process_text(doc) for doc in new_docs]
processed_new_docs[:3]

In [ ]:
# Step 2 - Reduce dimensionality
umap_model = UMAP(n_neighbors=15, n_components=15, min_dist=0.0, metric='cosine', random_state=1)

# Step 3 - Cluster reduced embeddings
hdbscan_model = HDBSCAN(min_cluster_size=15, metric='euclidean', cluster_selection_method='leaf', prediction_data=True)

# Step 4 - Tokenize topics
vectorizer_model = CountVectorizer(stop_words="english")

# Step 5 - Create topic representation
ctfidf_model = ClassTfidfTransformer()

In [ ]:
topic_model = BERTopic(
  embedding_model=embedding_model,
  umap_model=umap_model,
  hdbscan_model=hdbscan_model,
  vectorizer_model=vectorizer_model,
  ctfidf_model=ctfidf_model
)

topics, probs = topic_model.fit_transform(documents=processed_new_docs, embeddings=new_emb)

In [ ]:
topic_model.get_topic_info()

In [ ]:
fig = plt.figure(figsize=(15, 7))

topic_model.visualize_documents(new_docs, topics=[-1, 0, 1, 2, 3])

In [ ]:
topic_model.visualize_topics(topics=[-1, 0, 1, 2, 3])

In [ ]:
topic_model.get_representative_docs()

In [ ]:
topic_model.get_topics()

Исходя из представленных ключевых слов и репрезентативные документы, кажется, что кластеры сформированы довольно логично и адекватно отражают различные тематические области:

* **Кластер -1** `Ошибки, Проблемы с ОФД`
    - Этот кластер кажется сосредоточенным вокруг технических ошибок, связанных с облачным фискальным документооборотом (ОФД) и не только.
* **Кластер 0:** `Технические и операционные вопросы работы терминалов и касс`
    - Включает вопросы функционирования терминалов, проблемы при проведении безналичных оплат и вопросы работы кассовых аппаратов, включая комплексные системы, такие как Эвотор.
* **Кластер 1:** `Вопросы по оплате приложений`
    - Фокусируется на проблемах и запросах, связанных с процедурой оплаты в различных приложениях.
* **Кластер 2:** `Оплата сим-карт через терминалы и приложения`
    - Основная тематика вопросов касается процесса оплаты сим-карт, включая трудности при оплате через терминалы и мобильные приложения.
* **Кластер 3:** `Вопросы налогообложения, регистрации касс и системы патентов`
    - Включает сложные вопросы, связанные с изменением систем налогообложения, регистрацией кассовых аппаратов и применением патентной системы налогообложения, а также проблемы с задним числом и коррекцией записей.

**Итак, кластеризация первых сообщений пользовтелей выполнена и топики получены.**

**Чем я принебрег:**
* Не учитывал те сессии, где запрос пользователя не был четко понятен в первом сообщении. Например, первой сообщение: *"Добрый день"*
* Не учитывал взаимосвязь между сессиями

__! Так же важно, что это был срез запросов пользователей и полученные результаты могут быть не репрезентативными__

**Как можно попробовать улучшить:**
* Поэксперементировать с моделью для эмбеддингов
* Покрутить параметры для уменьшения размерности эмбеддингов и кластеризации, замеряя *Силуэтный коэффициент* (Я же просто руками перебрал некоторые параметры и глазами выбрал нормальное разбиение)
* Использовать бóльший объем данных

Тут я проверял гипотезу про сессии:

Смотрю по такой фразе и ищу где это первое сообщение в сессии

In [ ]:
df[df.text.str.contains("Спасибо, ответ помог")]

In [ ]:
df[df.session_id == "!PRkcIndqEDmvYkBSWz-1.0"]

In [ ]:
df[df.session_id == "!PRkcIndqEDmvYkBSWz-2.0"]

In [ ]:
df[df.session_id == "!PRkcIndqEDmvYkBSWz-3.0"]

Видно, что сессии `!PRkcIndqEDmvYkBSWz-2.0` пользователь перестает отвечать, видимо, решает проблему, а через час в сессии `!PRkcIndqEDmvYkBSWz-3.0` благодарит за помощь.

Из-за этого и получились странные результаты при кластеризации вначале

Также, кажется, что session_id сформирован так: f"{хэш юзера}-{номер обращения}"

Таким образом, уменьшая номер обращения, можно получать контекст из предыдущих сессий. Но здесь данные урезаны, поэтому не всегда мы получим предыдущие сессии